<a href="https://colab.research.google.com/github/jrleonett/visioncomputacional/blob/main/visioncomputacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de imágenes con YOLO y Visión por Computadora

Este proyecto utiliza el modelo YOLO (You Only Look Once) para procesar imágenes y detectar objetos en ellas. El código está dividido en tres fases principales: creación de una carpeta para almacenar resultados, procesamiento de la imagen y descarga de los resultados en un archivo ZIP. Realizado por **José R. Leonett** para la comunidad de peritos forenses digitales de Guatemala www.forensedigital.gt

In [ ]:
#@title Fase 1: Carga de variables y creación de la carpeta "EVIDENCIA"
import os
import cv2
import zipfile
from google.colab import files
from google.colab.patches import cv2_imshow

pip install mxnet gluoncv matplotlib

# Crear la carpeta "EVIDENCIA"
if not os.path.exists('EVIDENCIA'):
    os.makedirs('EVIDENCIA')

print("Carpeta 'EVIDENCIA' creada.")



In [ ]:
#@title Fase 2: Procesamiento de la imagen y detección de objetos
# Cargar YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Cargar la imagen
uploaded = files.upload()
image_name = next(iter(uploaded))
image = cv2.imread(image_name)

# Obtener dimensiones de la imagen
height, width, _ = image.shape

# Preprocesar la imagen para YOLO
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
detections = net.forward(output_layers)

# Procesar las detecciones
conf_threshold = 0.5
nms_threshold = 0.4
boxes = []
confidences = []
class_ids = []

for output in detections:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Aplicar Non-Maximum Suppression (NMS)
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# Dibujar las cajas y etiquetas en la imagen
for i in indices:
    i = i[0]
    box = boxes[i]
    x, y, w, h = box
    label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Mostrar la imagen con las detecciones
cv2_imshow(image)

# Guardar la imagen procesada en la carpeta "EVIDENCIA"
output_image_path = os.path.join('EVIDENCIA', 'imagen_procesada.jpg')
cv2.imwrite(output_image_path, image)
print(f"Imagen procesada guardada en {output_image_path}")

In [ ]:
#@title Fase 3: Descargar los resultados en un archivo ZIP
# Crear un archivo ZIP con los resultados
zip_path = 'EVIDENCIA.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk('EVIDENCIA'):
        for file in files:
            zipf.write(os.path.join(root, file))

# Descargar el archivo ZIP
files.download(zip_path)
print(f"Archivo {zip_path} descargado.")

# Zona de resultados